In [1]:
import pandas as pd
import math

In [2]:
class PickListMaker:
    
    def __init__(self, filenames, solution_moles, solution_volume, \
                 buffer_moles, scaffold="M13mp18", ratio="10:1"):
        self.scaffold_molarity = {"M13mp18": 0.21e-12/1e-6}
        self.strands_df = self.generates_dataframes(filenames)
        self.solution_moles = solution_moles
        self.solution_volume = solution_volume
        self.buffer_moles = buffer_moles
        self.scaffold = scaffold
        self.ratio = float(ratio.split(":")[0])
        self.picklist_df = self.generate_picklist(self.strands_df)

    def generates_dataframes(self, filenames):
        strands_dfs = []
        for filename in filenames:
            file_extn = filename.split(".")[-1]
            if file_extn == "xlsx": 
                strands_dfs.append(pd.read_excel(filename))
            elif file_extn == "csv":
                strands_dfs.append(pd.read_csv(filename))
        return strands_dfs

    def generate_picklist(self, all_staples):
        source_plate_name = []
        source_plate_type = []
        source_well_positions = []
        sample_comments = []
        destination_plate_name = []
        destination_wells = []
        transfer_volumes = []
        scaffold_transfer_volume = (self.solution_moles * self.solution_volume) / self.scaffold_molarity[self.scaffold]
        staple_concentrations = [staples_group["Measured Concentration µM "].mean() for staples_group in all_staples]
        staples_num = sum([len(staples_group) for staples_group in all_staples])
        staple_concentrations_avg = math.floor(sum(staple_concentrations) / len(staple_concentrations))*1e-6
        staple_transfer_volume = (self.ratio * 
                                  self.solution_moles * self.solution_volume) / staple_concentrations_avg
        staple_transfer_volume = int(round(staple_transfer_volume*1e+9, 0)) # in nanoliters
        dna_volume = scaffold_transfer_volume + staples_num * staple_transfer_volume*1e-9
        buffer_volume = self.solution_volume - dna_volume
        magnesium_volume = self.solution_volume * self.buffer_moles
        print("Average Staple Concetration is: {:.1f}µl with transfer volume {:.1f}.".format(round(staple_concentrations_avg*1e+6, 0), staple_transfer_volume))
        print("Total DNA volume is {:.2f}µl and buffer volume is {:.2f}µl.".format(dna_volume*1e+6, buffer_volume*1e+6))
        print("Magnesium volume is {:.2f}µl. This will be placed in the last available well.".format(magnesium_volume*1e+6))
        for idx, staple_pkg in enumerate(all_staples):
            for item, row in staple_pkg.iterrows():
                source_plate_name.append(f"Source[{idx+1}]")
                source_plate_type.append("384PP_AQ_BP")
                last_well_position = row["Well Position"]
                source_well_positions.append(last_well_position)
                sample_comments.append(row["Sequence"])
                destination_plate_name.append("Destination[1]")
                destination_wells.append("I10")
                transfer_volumes.append(staple_transfer_volume)
                
        source_plate_name.append("Source[3]")
        source_plate_type.append("384PP_AQ_BP")
        magnesium_well_position = last_well_position
        
        if int(magnesium_well_position[1:]) > 24:
            letter_ord = (ord(s.upper())+1 - 65) % 26 + 65
            if letter_ord > 80:
                print("Ran out of well to place buffer, please pipet manually.")
            else:
                magnesium_well_position[0] = chr(letter_ord)
        else:
            magnesium_well_position = magnesium_well_position[0] + str(int(magnesium_well_position[1:]) + 1).zfill(2)
                                            
            
        if magnesium_well_position != last_well_position:
            source_well_positions.append(magnesium_well_position)
            sample_comments.append("Mg")
            destination_plate_name.append("Destination[1]")
            destination_wells.append("I10")
            transfer_volumes.append(magnesium_volume*1e+9)
            
        echo_df = pd.DataFrame({"Source Plate Name": source_plate_name, "Source Plate Type": source_plate_type, \
                           "Source Well": source_well_positions, "Sample Comments": sample_comments, \
                           "Destination Plate Name": destination_plate_name, "Destination Well": destination_wells, \
                           "Transfer Volume": transfer_volumes})
        echo_df["Transfer Volume"] = echo_df["Transfer Volume"].astype('int')
        echo_df = echo_df.set_index("Source Plate Name")
        return echo_df

In [3]:
picklist = PickListMaker(["~/Downloads/samson_plates.csv", "~/Downloads/Plate Specs.xlsx"], \
                        10e-09, 50e-06, 10e-3)

Average Staple Concetration is: 201.0µl with transfer volume 25.0.
Total DNA volume is 8.23µl and buffer volume is 41.77µl.
Magnesium volume is 0.50µl. This will be placed in the last available well.


In [ ]:
picklist.picklist_df

In [ ]:
"10:1".split(":")